# Connecting to a MS SQL database

The objective of this POC project is to:

1) To establish a successful MS SQl db connection using python.

2) Successfully excute CRUD operations.

# Set up my sql server using docker [Reference](https://learn.microsoft.com/en-us/sql/linux/quickstart-install-connect-docker?view=sql-server-ver16&pivots=cs1-bash).

#### Steps:

1) Pull the SQL Server 2022 (16.x) Linux container image from the Microsoft Container Registry.

```sh
docker pull mcr.microsoft.com/mssql/server:2022-latest
```

2) To run the Linux container image with Docker, you can use the following command from a bash shell or elevated PowerShell command prompt.

```sh
docker run -e "ACCEPT_EULA=Y" -e "MSSQL_SA_PASSWORD=Admin@101" \
   -p 1433:1433 --name sql1 --hostname sql1 \
   -d \
   mcr.microsoft.com/mssql/server:2022-latest
```

3) To view your Docker containers, use the docker ps command.

```sh
docker ps -a
```

4) Check SQL server logs

```sh 
docker exec -t sql1 cat /var/opt/mssql/log/errorlog | grep connection
```

5) connect to sql server

```sh
docker exec -it sql1 "bash"
```

6) Connect to db using the bash terminal of docker container.

```sh
/opt/mssql-tools/bin/sqlcmd -S localhost -U SA -P "Admin@101"
```

7) Install SQL ODBC Driver and SQL Command tools.

```sh

if ! [[ "18.04 20.04 22.04" == *"$(lsb_release -rs)"* ]];
then
    echo "Ubuntu $(lsb_release -rs) is not currently supported.";
    exit;
fi

sudo su
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -

curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list > /etc/apt/sources.list.d/mssql-release.list

exit
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get install -y msodbcsql18
# optional: for bcp and sqlcmd
sudo ACCEPT_EULA=Y apt-get install -y mssql-tools18
echo 'export PATH="$PATH:/opt/mssql-tools18/bin"' >> ~/.bashrc
source ~/.bashrc
# optional: for unixODBC development headers
sudo apt-get install -y unixodbc-dev

```

# Setting up sample database

1) Login to docker conrtainer as root user

```sh
docker exec -u 0 -it sql1 bash
```

2) Download the AdventureWorks2019 sample database bakup file into the docker container

```sh
wget https://github.com/Microsoft/sql-server-samples/releases/download/adventureworks/AdventureWorks2019.bak -O /var/opt/mssql/backup/AdventureWorks2019.bak
```

3) Restore database using the backup file

```sql
USE [master]
RESTORE DATABASE [AdventureWorks2019]
FROM DISK = '/var/opt/mssql/backup/AdventureWorks2019.bak'
WITH MOVE 'AdventureWorks2017' TO '/var/opt/mssql/data/AdventureWorks2019.mdf',
MOVE 'AdventureWorks2017_log' TO '/var/opt/mssql/data/AdventureWorks2019_log.ldf',
FILE = 1,  NOUNLOAD,  STATS = 5
GO
```

# Connect to MS SQL Server using the `pyodbc`

### Install ```pyodbc```

In [1]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


### Connect to server

In [4]:
import pyodbc 
# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
server = 'localhost,1433' 
database = 'TestDB' 
username = 'SA' 
password = 'Admin@101' 
# ENCRYPT defaults to yes starting in ODBC Driver 18. It's good to always specify ENCRYPT=yes on the client side to avoid MITM attacks.
cnxn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';ENCRYPT=no;UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

### Run Query

In [5]:
#Sample select query
cursor.execute("SELECT @@version;") 
row = cursor.fetchone() 
while row: 
    print(row[0])
    row = cursor.fetchone()

Microsoft SQL Server 2022 (RTM) - 16.0.1000.6 (X64) 
	Oct  8 2022 05:58:25 
	Copyright (C) 2022 Microsoft Corporation
	Developer Edition (64-bit) on Linux (Ubuntu 20.04.5 LTS) <X64>


### Insert a Row

In [11]:
#Sample insert query
count = cursor.execute("USE TestDB;").
cnxn.commit()
print(count)

-1
